In [1]:
import re
"""
此类用于转换为前序表达式，用以构建公式树
"""

class Formula:
    def __init__(self, input):
        """   
        input 为谓词公式，
        其中'¬' '∧' '∨' '→' '∀' '∃'为演算符
        '(' ')' 为更改运算优先级符号
        '['']' 为谓词符号，如Q[x,y]
        """
        self.source = input
        self.to_list_result = None
        self.forward = None  # 存储前序表达式

    def to_list(self):
        """
        将输入的谓词公式转化为列表。
        1. 量词符号与下一个操作数作为一个元素
        2. 谓词符号作为一个元素
        3. 其他符号（运算符、括号）分别作为一个元素
        如：∀x (P(x) → ∃y Q(x, y))变为['∀x', '(', 'P(x)', '→', '∃y', 'Q(x, y)']
        """
        expression = self.source
        result = []
        
        # 正则表达式用于匹配不同的元素
        pattern = r'(\∀\w+|\∃\w+|[A-Za]+(?:\[[^\[\]]*\])*|[¬∧∨→()])'
        
        # 使用正则匹配所有符号
        tokens = re.findall(pattern, expression)
        
        # 处理量词与操作数合并的情况
        for token in tokens:
            # if token.startswith('∀') or token.startswith('∃'):
            #     # 量词符号和操作数合并为一个元素
            #     result.append(token)
            # elif re.match(r'[A-Za-z]+(?:\[[^\[\]]*\])*', token):
            #     # 谓词符号（如P(x), Q(x, y)）
            #     result.append(token)
            # elif token in ['¬', '∧', '∨', '→', '(', ')']:
            #     # 其他运算符和括号
                result.append(token)
        
        self.to_list_result = result
        return result

    def to_forward(self):
        """
        将列表内元素转化为前序表达式，并更新self.forward。
        1. 运算符与操作数的元素算作一个操作数元素
        2. 以列表形式输出前序表达式
        """
        # 调用 to_list 方法，获取转化后的列表
        if not self.to_list_result:
            self.to_list()

        # 反转输入的列表，确保运算符和操作数按照前序的顺序处理
        expr_list = self.to_list_result[::-1]


        # 前序表达式栈
        stack = []
        operators = ['¬', '∧', '∨', '→']
        precedence = {'¬': 4, '∧': 3, '∨': 2, '→': 1}
        result=[]
        # 转换为前序表达式的逻辑
        for element in expr_list:
            # print(element)
            if element not in operators and element not in ['(', ')']:  # 如果是操作数或谓词符号
                result.insert(0,element)
            elif element == ')':
                stack.append(element)
            elif element == '(':
                # 当遇到右括号时，弹出栈中的元素直到遇到左括号

                while stack and stack[-1] != ')':
                    result.insert(0,stack.pop(-1))
                stack.pop()  # 弹出左括号
            else:  # 如果是运算符
                while (stack and stack[-1] in operators and 
                       precedence[element] < precedence[stack[-1]]):
                    result.insert(0,stack.pop(-1))
                stack.append(element)

        while len(stack)!=0:
            result.insert(0,stack.pop(0))
        self.forward = result
        return self.forward



输入定义
1. 大写字母仅用作公式声明 如Q[x,y]
2. 小写字母仅用作公式内部变量和量词声明变量，如∃y，Q[x,y]
3. 公式用[]限定范围，如有嵌套内部使用()，如F[T(x,y),Z(c,m)]

In [2]:

if __name__ == "__main__":
    formula_3=Formula("(∀x)(P[x] → ¬(∀y)(P[y] →P[f(x,y)])) ∧ (¬(∀y)(Q[x,y] → P[y]))")
    result_3=formula_3.to_forward()
    print(result_3)


['∧', '∀x', '→', 'P[x]', '¬', '∀y', '→', 'P[y]', 'P[f(x,y)]', '¬', '∀y', '→', 'Q[x,y]', 'P[y]']


In [3]:
import os
import copy
import copy
"""
本类用于构建树
"""

class Tree:
    """
    传入前序表达式列表
    """
    def __init__(self, for_formula):
        self.for_formula = for_formula
        self.source = copy.deepcopy(for_formula)  
        self.head = None
        self.current = None

    def __repr__(self):
        return self.__hash__()

    def build_tree(self, current_node=None):
        # 当公式为空时返回
        if len(self.for_formula) == 0:
            return

        if self.head is None:
            # 根节点初始化
            self.head = Node(element='start')
            current_node = self.head
            self.build_tree(current_node)

        current_element = current_node.element

        # 处理量词
        if current_element.startswith('∀') or current_element.startswith('∃'):
            if len(self.for_formula) > 0:
                new_element = self.for_formula.pop(0)
                new_node = Node(new_element, parent=current_node)
                current_node.left_child = new_node
                self.build_tree(new_node)
        elif current_element == 'start':
            if len(self.for_formula) > 0:
                new_element = self.for_formula.pop(0)
                new_node = Node(new_element, parent=current_node)
                current_node.left_child = new_node
                self.build_tree(new_node)

        # 处理逻辑操作符
        elif current_element in ['∧', '∨', '→']:
            if len(self.for_formula) > 1:
                new_left_element = self.for_formula.pop(0)
                new_left_node = Node(new_left_element, parent=current_node)
                current_node.left_child = new_left_node
                self.build_tree(new_left_node)  

                new_right_element = self.for_formula.pop(0)
                new_right_node = Node(new_right_element, parent=current_node)
                current_node.right_child = new_right_node
                self.build_tree(new_right_node)

        # 处理否定运算符
        elif current_element == '¬':
            if len(self.for_formula) > 0:
                new_element = self.for_formula.pop(0)
                new_node = Node(new_element, parent=current_node)
                current_node.left_child = new_node
                self.build_tree(new_node)

        else:
            # 如果是命题变量或无操作符的节点，标记为叶节点
            current_node.is_leaf = True

# 测试用例



    def visualize(self, node=None,parent=None, graph=None):
        """
        输出Mermaid格式的树，并保存为文件
        """
        if node is None:
            node = self.head
        if graph is None:
            graph = ['graph TD']

        node_name = f"Node{node.__hash__()}"

        # 节点格式
        graph.append(f"    {node_name}[\"{node.element}\"]")

        # 处理父子连接
        if parent:
            graph.append(f"    {parent} --> {node_name}")

        # 递归绘制子节点
        if node.left_child:
            self.visualize(node.left_child, node_name, graph)
        if node.right_child:
            self.visualize(node.right_child, node_name, graph)

        return graph

    def save_mermaid(self):
        """
        生成Mermaid格式图，并保存为文件
        """
        graph = self.visualize()
        filename = f"{'_'.join(self.source)}.mmd"  # 文件名为表达式的source名称
        file_path = os.path.join(os.getcwd(), filename)

        with open(file_path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(graph))
        print(f"Mermaid格式文件已保存为: {file_path}")

    def show_tree(self):
        """
        展示树并保存为mermaid文件
        """
        self.save_mermaid()



class Node:
    def __init__(self, element, left_child=None, right_child=None, parent=None):
        self.element = element
        self.left_child = left_child
        self.right_child = right_child
        self.parent = parent
        self.is_leaf = False
    
    def __str__(self):
        return self.element
    def __repr__(self):
        return self.element


In [4]:
# 测试代码
if __name__ == "__main__":
    # 输入的前序表达式


    formula = [ '¬','∧', '∀x', '→', 'P(x)', '∧', 'Q(x)', '∃y', '∨', 'R(y)', 'S(y)', '∃z', '→', 'T(z)', '∀w', '∧', 'U(w)', 'V(w)']
    formula_2=['∀x', '→', 'P(x)', '∃y', 'Q(x, y)']
    formula_3=['∧', '∀x', '→', 'P[x]', '¬', '∀y', '→', 'P[y]', 'P[F(x,y)]', '¬', '∀y', '→', 'Q[x,y]', 'P[y]']
    


    tree_3 = Tree(formula_3)
    tree_3.build_tree()
    

    """
    生成文件需要支持mermaid设备
    """
    tree_3.show_tree()



Mermaid格式文件已保存为: d:\project\ai_work\6\∧_∀x_→_P[x]_¬_∀y_→_P[y]_P[F(x,y)]_¬_∀y_→_Q[x,y]_P[y].mmd


In [5]:
def eliminate_implication(node):
    """消去蕴涵符号(A → B)变为(¬A ∨ B)"""
    if node is None:
        return

    if node.element == '→':
        # 替换当前节点为“∨”
        node.element = '∨'

        # 左子树设置为“¬”节点
        new_left_node = Node('¬', parent=node)
        temp = node.left_child
        node.left_child = new_left_node
        new_left_node.left_child = temp

        # 更新子节点的 parent 属性
        if temp:
            temp.parent = new_left_node

    # 递归处理左子树和右子树
    if node.left_child:
        eliminate_implication(node.left_child)
    if node.right_child:
        eliminate_implication(node.right_child)


eliminate_implication(tree_3.head)
tree_3.show_tree()


Mermaid格式文件已保存为: d:\project\ai_work\6\∧_∀x_→_P[x]_¬_∀y_→_P[y]_P[F(x,y)]_¬_∀y_→_Q[x,y]_P[y].mmd


In [6]:
def sink_negation(node):
    """
    非符号（¬）下沉操作，支持量词（∀、∃）节点的识别与处理。
    遵循逻辑等价规则（De Morgan定律）和量词的变换。
    """
    if node is None:
        return

    # 如果当前节点是"¬"，处理其子树
    if node.element == '¬':

        
        child = node.left_child  # ¬节点只有左子树

        # if child.is_leaf:
        #     child.parent=node.parent
        #     node.parent.left_child=child
        #     child.element='¬'+child.element
        if child is None:
            return  # 如果子节点为空，直接返回

        # 处理量词节点
        if child.element.startswith('∀'):
            # ¬∀xP(x) -> ∃x¬P(x)
            node.element = '∃' + child.element[1:]  # 替换量词
            node.left_child = Node('¬', parent=node)  # 新建¬节点
            node.left_child.left_child = child.left_child  # 接管子树
            if node.left_child.left_child:
                node.left_child.left_child.parent = node.left_child

        elif child.element.startswith('∃'):
            # ¬∃xP(x) -> ∀x¬P(x)
            node.element = '∀' + child.element[1:]  # 替换量词
            node.left_child = Node('¬', parent=node)  # 新建¬节点
            node.left_child.left_child = child.left_child  # 接管子树
            if node.left_child.left_child:
                node.left_child.left_child.parent = node.left_child

        # 处理逻辑符号节点
        elif child.element == '∧':
            # ¬(A ∧ B) -> (¬A ∨ ¬B)
            node.element = '∨'
            left_neg = Node('¬', parent=node)
            right_neg = Node('¬', parent=node)

            left_neg.left_child = child.left_child
            right_neg.left_child = child.right_child

            node.left_child = left_neg
            node.right_child = right_neg

            if left_neg.left_child:
                left_neg.left_child.parent = left_neg
            if right_neg.left_child:
                right_neg.left_child.parent = right_neg

        elif child.element == '∨':
            # ¬(A ∨ B) -> (¬A ∧ ¬B)
            node.element = '∧'
            left_neg = Node('¬', parent=node)
            right_neg = Node('¬', parent=node)

            left_neg.left_child = child.left_child
            right_neg.left_child = child.right_child

            node.left_child = left_neg
            node.right_child = right_neg

            if left_neg.left_child:
                left_neg.left_child.parent = left_neg
            if right_neg.left_child:
                right_neg.left_child.parent = right_neg

        elif child.element == '¬':
            # ¬(¬A) -> A
            node.element = child.left_child.element
            node.left_child = child.left_child.left_child
            node.right_child = child.left_child.right_child

            if node.left_child:
                node.left_child.parent = node
            if node.right_child:
                node.right_child.parent = node

    # 递归处理左右子树
    if node.left_child:
        sink_negation(node.left_child)
    if node.right_child:
        sink_negation(node.right_child)
sink_negation(tree_3.head)
tree_3.show_tree()

Mermaid格式文件已保存为: d:\project\ai_work\6\∧_∀x_→_P[x]_¬_∀y_→_P[y]_P[F(x,y)]_¬_∀y_→_Q[x,y]_P[y].mmd


In [850]:
a=tree_3.head.left_child.right_child.left_child.right_child.left_child
print(a.is_leaf)

False


In [851]:


# def merge(node):
#     if node is None:
#         return
#     child = node.left_child 

#     # 如果当前节点是"¬"，处理其子树
#     if node.element == '¬' and child.is_leaf:
#             child.parent=node.parent
#             if node.parent.right_child==node:
#                 node.parent.right_child=child
#             node.parent.left_child=child
#             child.element='¬'+child.element
#     else:
#         merge(node.left_child)
#         merge(node.right_child)

# merge(tree_3.head)
# tree_3.show_tree()


In [ ]:
def change(node, x, y):
    if node is None:
        print("空节点")
        return
    if node.element =='start':
        change(node.left_child,x,y)
    if node.element in ['¬', '∧', '∨']:
        if node.left_child:
            change(node.left_child, x, y)
        if node.right_child:
            change(node.right_child, x, y)
    elif node.element.startswith('∀') or node.element.startswith('∃'):
        node.element = node.element.replace(x, y)
        if node.left_child:
            change(node.left_child, x, y)
    else:
        node.element = node.element.replace(x, y)

def VariableStandardization(node, lis=None):

    """
    替换量词辖域内所有变量
    """
    if lis is None:
        lis = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    if node is None:
        return
    if node.element=='start':
        VariableStandardization(node.left_child)
    if node.element.startswith('∀') or node.element.startswith('∃'):
        x = node.element[1:]
        change(node, x, lis.pop(0))
        VariableStandardization(node.left_child, lis)
    elif node.element in ['¬', '∧', '∨']:
        if node.left_child:
            VariableStandardization(node.left_child, lis)
        if node.right_child:
            VariableStandardization(node.right_child, lis)
    # VariableStandardization(node.left_child, lis)

VariableStandardization(tree_3.head)    
tree_3.show_tree()




Mermaid格式文件已保存为: d:\project\ai_work\6\∧_∀x_→_P[x]_¬_∀y_→_P[y]_P[F(x,y)]_¬_∀y_→_Q[x,y]_P[y].mmd


In [ ]:


def eliminate_exists_quantifier(node, stack, counter):
    """
    使用栈消去存在量词，并替换为数字列表。
    """
    if node is None:
        return None, counter

    # 如果当前节点是存在量词
    if node.element.startswith('∃'):
        # 1. 存在量词在全称量词内的情况
        if stack and stack[-1] == '∀':  

            counter += 1
            new_element = f"F{counter}({node.element[1:]})"
            new_element.replace(stack[-1][1:],f'{counter}({node.element[1:]})')
            node.element = new_element
        else:
            # 2. 存在量词不在全称量词内，直接删除量词
            node.element = f"F{counter}" # 或者用常量替代
            return None, counter  # 删除该节点

    # 递归处理子节点
    node.left_child, counter = eliminate_exists_quantifier(node.left_child, stack, counter)
    node.right_child, counter = eliminate_exists_quantifier(node.right_child, stack, counter)

    # 返回更新后的节点和counter
    return node, counter

def process_quantifiers(node, stack=None, counter=None):
    """
    处理树中的量词，使用栈来跟踪量词的嵌套结构。
    """
    if stack is None:
        stack = []
    if counter is None:
        counter = 0
    if node is None:
        return node, counter

    # 如果是全称量词，入栈
    if node.element.startswith('∀'):
        stack.append(node)

    # 如果是存在量词，处理消去
    if node.element.startswith('∃'):
        node, counter = eliminate_exists_quantifier(node, stack, counter)

    # 遍历左子树和右子树
    if node.left_child:
        node.left_child, counter = process_quantifiers(node.left_child, stack, counter)
    if node.right_child:
        node.right_child, counter = process_quantifiers(node.right_child, stack, counter)

    # 如果是量词，出栈
    if node.element.startswith('∀') or node.element.startswith('∃'):
        stack.pop()

    return node, counter


head_node, counter = process_quantifiers(tree_3.head)


In [8]:

def toPrenexNormalForm(node):

    """
    消去全称量词
    """
    if node is None:
        return
    
    # 处理当前节点的量词
    if node.element.startswith('∀') or node.element.startswith('∃'):
        if node.parent:
            # 将当前量词的子节点连接到父节点
            if node.parent.left_child == node:
                node.parent.left_child = node.left_child
            else:
                node.parent.right_child = node.left_child
            # 更新子节点的 parent
            if node.left_child:
                node.left_child.parent = node.parent
        else:
            # 若当前节点是根节点，直接更新根为其子节点
            if node.left_child:
                node.left_child.parent = None
                node = node.left_child

    # 递归处理左子树和右子树
    if node.left_child:
        toPrenexNormalForm(node.left_child)
    if node.right_child:
        toPrenexNormalForm(node.right_child)


toPrenexNormalForm(tree_3.head)
tree_3.show_tree()
    

Mermaid格式文件已保存为: d:\project\ai_work\6\∧_∀x_→_P[x]_¬_∀y_→_P[y]_P[F(x,y)]_¬_∀y_→_Q[x,y]_P[y].mmd


In [9]:
def toConjunctiveNormalForm(node):
    """
    交换律
    """
    # 递归终止条件：当节点为空或为叶子节点时，直接返回
    if node is None:
        return node

    # 递归遍历左子树和右子树
    node.left_child = toConjunctiveNormalForm(node.left_child)
    node.right_child = toConjunctiveNormalForm(node.right_child)

    # 如果当前节点是析取（∨），应用分配律
    if node.element == '∨':
        if node.left_child and node.left_child.element == '∧':  # (A ∧ B) ∨ C => (A ∨ C) ∧ (B ∨ C)
            # 创建新的左右子树
            new_left = Node('∨', left_child=node.left_child.left_child, right_child=node.right_child, parent=node)
            new_right = Node('∨', left_child=node.left_child.right_child, right_child=node.right_child, parent=node)
            
            # 更新节点父子关系
            node.left_child = new_left
            node.right_child = new_right
            node.element = '∧'  # 更新当前节点为合取

        elif node.right_child and node.right_child.element == '∧':  # A ∨ (B ∧ C) => (A ∨ B) ∧ (A ∨ C)
            # 创建新的左右子树
            new_left = Node('∨', left_child=node.left_child, right_child=node.right_child.left_child, parent=node)
            new_right = Node('∨', left_child=node.left_child, right_child=node.right_child.right_child, parent=node)
            
            # 更新节点父子关系
            node.left_child = new_left
            node.right_child = new_right
            node.element = '∧'  # 更新当前节点为合取

    # 返回更新后的节点
    return node

# 调用函数并显示结果
toConjunctiveNormalForm(tree_3.head)
tree_3.show_tree()


Mermaid格式文件已保存为: d:\project\ai_work\6\∧_∀x_→_P[x]_¬_∀y_→_P[y]_P[F(x,y)]_¬_∀y_→_Q[x,y]_P[y].mmd


In [ ]:
def split(node, result=None):
    """
    分割子句集，有bug
    """
    if result is None:
        result = []

    element = node.element
    print(result)
    # 如果是合取符号（∧），递归处理左右子树
    if element == '∧':
        split(node.left_child, result)
        split(node.right_child, result)
    
    # 如果是析取符号（∨），收集节点
    elif element == '∨':
        result.append(collect(node))

    return result

def collect(node):
    # 处理左子节点是非符号（¬）的情况
    if node.left_child and node.left_child.element == '¬':
        left_string = node.left_child.element + node.left_child.left_child.element
    elif node.left_child:
        left_string=node.left_child.element

    # 处理右子节点是非符号（¬）的情况
    if node.right_child and node.right_child.element == '¬':
        right_string = node.right_child.element + node.right_child.left_child.element
    elif node.right_child:
        right_string = node.right_child.element


    # 拼接并返回形式为 'A ∨ B' 或 '¬A ∨ ¬B'
    string = left_string + node.element + right_string
    return string


# 示例调用
result = split(tree_3.head)
print(result)



[]
[]
